In [1]:
from keras.datasets import fashion_mnist
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from sklearn.model_selection import train_test_split

In [2]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [3]:
np.random.seed(0)

In [11]:
class FFNNClass:   
    def __init__(self, hidden_sizes=[3], algo='SGD', initialization='xavier', activation='sigmoid', loss='cross_entropy'):
        #self.nx = n_inputs
        #self.ny = n_outputs
        self.nx = 784
        self.ny = 10
        self.nh = len(hidden_sizes)
        self.sizes = [self.nx] + hidden_sizes + [self.ny] 
        self.algo = algo
        self.params={}
        #self.gradients={}
        self.dW = {}
        self.dB = {}
        self.dH = {}
        self.dA = {}
        self.update_params={}
        self.prev_update_params={}
        self.initialization = initialization
        self.activation = activation
        self.loss = loss
        for i in range(self.nh+1):
            self.update_params["v_w"+str(i+1)]=0
            self.update_params["v_b"+str(i+1)]=0
            self.update_params["m_b"+str(i+1)]=0
            self.update_params["m_w"+str(i+1)]=0
            self.prev_update_params["v_w"+str(i+1)]=0
            self.prev_update_params["v_b"+str(i+1)]=0
            
        if initialization=='xavier':
            for i in range(self.nh+1):
                self.params["W"+str(i+1)]= np.random.randn(self.sizes[i], self.sizes[i+1]) * np.sqrt(1/(self.sizes[i]+self.sizes[i+1]))
                self.params["B"+str(i+1)]= np.zeros((1, self.sizes[i+1]))
        elif initialization=='random':
            for i in range(self.nh+1):
                self.params["W"+str(i+1)]= np.random.randn(self.sizes[i], self.sizes[i+1]) * 0.01
                self.params["B"+str(i+1)]= np.zeros((1, self.sizes[i+1]))
            
    def sigmoid(self, x):
        #return np.where(x >= 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))
        x = np.clip(x, -500, 500)
        return 1.0/(1.0 + np.exp(-x))
    def grad_sigmoid(self, x):
        return x*(1-x)
    
    def relu(self, x):
        return np.maximum(0,x)
    def grad_relu(self, x):
        return 1*(x>0)
    
    def tanh(self, x):
        return np.tanh(x)
    def grad_tanh(self, x):
        return (1 - (np.tanh(x)**2))

    def softmax(self, x):
        eps = 1e-9
        exps = np.exp(x)
        return exps / (np.sum(exps,axis=1).reshape(-1,1) + eps)

    def forward_pass(self, x, params=None):
        if params is None:
            params=self.params
        self.A = {}
        self.H = {}
        #self.H[0] = x.reshape(1, -1)
        self.H[0] = x  #dimension is 16 * 784
        for i in range(self.nh):
            self.A[i+1] = np.matmul(self.H[i], params["W"+str(i+1)]) + params["B"+str(i+1)]
            if self.activation == 'sigmoid':
                self.H[i+1] = self.sigmoid(self.A[i+1])
            elif self.activation == 'tanh':
                self.H[i+1] = self.tanh(self.A[i+1])
            elif self.activation == 'relu':
                self.H[i+1] = self.relu(self.A[i+1])
        self.A[self.nh+1] = np.matmul(self.H[self.nh], params["W"+str(self.nh+1)]) + params["B"+str(self.nh+1)]
        self.H[self.nh+1] = self.softmax(self.A[self.nh+1])
        return self.H[self.nh+1]

    def predict(self, X):
        Y_pred = self.forward_pass(X)
        return np.array(Y_pred).squeeze()
    
    def compute_loss(self, Y, Y_hat, batch_size, lamb):
        eps = 1e-8
        if self.loss == 'cross_entropy':
            L = (-1.0 * np.sum(np.multiply(Y, np.log(Y_hat + eps))))/batch_size
        elif self.loss == 'mse':
            L = (1/2) * np.sum((Y-Y_hat)**2)/batch_size

        #Add L2 regularisation
        acc = 0
        for i in range(1, self.nh + 1):
            acc += np.sum(self.params["W"+str(i)]**2)

        L = L + (lamb/(2*batch_size))*acc

        return L

    def grad(self, x, y, params=None):
        if params is None:
            params=self.params
        self.forward_pass(x,params)
        m = x.shape[0]
        L = self.nh + 1
        if self.loss == 'cross_entropy':
            self.dA[L] = (self.H[L] - y)
        elif self.loss == 'mse':
            self.dA[L] = (self.H[L] - y) * self.H[L] * (1-self.H[L])
        for k in range(L, 0, -1):
            self.dW[k] = np.matmul(self.H[k-1].T, self.dA[k])
            self.dB[k] = np.sum(self.dA[k],axis=0).reshape(1,-1)
            self.dH[k-1] = np.matmul(self.dA[k], params["W"+str(k)].T)
            if self.activation == 'sigmoid':
                self.dA[k-1] = np.multiply(self.dH[k-1], self.grad_sigmoid(self.H[k-1]))
            elif self.activation == 'tanh':
                self.dA[k-1] = np.multiply(self.dH[k-1], self.grad_tanh(self.H[k-1]))
            elif self.activation == 'relu':
                self.dA[k-1] = np.multiply(self.dH[k-1], self.grad_relu(self.H[k-1]))
          
    def plot_loss_curve(self,train_costs, val_costs):
        '''
        Plots the train and validation cost curves
        '''
        plt.plot(train_costs.values(), 'r', label="Training loss")
        plt.plot(val_costs.values(), 'lime', label="validation loss")
        plt.title("Training and Validation Loss vs Number of Epochs", size=14)
        plt.xlabel("Number of epochs", size=14)
        plt.ylabel("Loss", size=14)
        plt.grid()
        plt.legend()
        plt.show()

    def fit(self, X, Y, X_VAL, Y_VAL,  epochs = 5, eta=0.01, gamma=0.9, eps=1e-8, beta=0.9, beta1=0.9, beta2=0.9, mini_batch_size=1,wt_decay=0.0):

        #if display_loss:
        train_loss = {}
        val_loss = {}
        train_acc = {}
        val_acc = {}
        M = X.shape[0]
        Mval = X_VAL.shape[0]
                
        if self.algo == "GD":
            for epoch in range(epochs):
                m = X.shape[0]
                self.grad(X,Y)
                for i in range(self.nh+1):
                    self.params["W"+str(i+1)] -= (eta * (self.dW[i+1]/m) + eta*wt_decay*(self.dW[i+1]/m))
                    self.params["B"+str(i+1)] -= eta * (self.dB[i+1]/m)
                #if display_loss:
                Y_pred = self.predict(X)
                Y_pred_val = self.predict(X_VAL)
                Y_temp = np.argmax(Y_pred,1)
                Y_val_temp = np.argmax(Y_pred_val,1)
                train_acc[epoch] = accuracy_score(y_train, Y_temp)
                val_acc[epoch] = accuracy_score(y_val, Y_val_temp)
                full_output = self.forward_pass(X)
                cost = self.compute_loss(Y, full_output, M, wt_decay)
                train_loss[epoch] = cost
                
                # Mean loss for the validation set
                out = self.forward_pass(X_VAL)
                val_cost = self.compute_loss(Y_VAL, out, Mval, wt_decay)
                val_loss[epoch] = val_cost

        elif self.algo == "SGD":
            for epoch in range(epochs):
                m = X.shape[0]
                for k in range(0,m,mini_batch_size):
                    self.grad(X[k:k+mini_batch_size], Y[k:k+mini_batch_size])
                    for i in range(self.nh+1):
                        self.params["W"+str(i+1)] -= (eta * (self.dW[i+1]/mini_batch_size) + eta*wt_decay*(self.dW[i+1]/mini_batch_size))
                        self.params["B"+str(i+1)] -= eta * (self.dB[i+1]/mini_batch_size)
                #if display_loss:
                Y_pred = self.predict(X)
                Y_pred_val = self.predict(X_VAL)
                Y_temp = np.argmax(Y_pred,1)
                Y_val_temp = np.argmax(Y_pred_val,1)
                train_acc[epoch] = accuracy_score(y_train, Y_temp)
                val_acc[epoch] = accuracy_score(y_val, Y_val_temp)

                full_output = self.forward_pass(X)
                cost = self.compute_loss(Y, full_output, M, wt_decay)
                train_loss[epoch] = cost
                
                # Mean loss for the validation set
                out = self.forward_pass(X_VAL)
                val_cost = self.compute_loss(Y_VAL, out, Mval, wt_decay)
                val_loss[epoch] = val_cost

                        
        elif self.algo == "Momentum":
            for epoch in range(epochs):
                m = X.shape[0]
                for k in range(0,m,mini_batch_size):
                    self.grad(X[k:k+mini_batch_size], Y[k:k+mini_batch_size])
                #self.grad(X, Y)
                    for i in range(self.nh+1):
                        self.update_params["v_w"+str(i+1)] = gamma *self.update_params["v_w"+str(i+1)] + eta * (self.dW[i+1]/mini_batch_size)
                        self.update_params["v_b"+str(i+1)] = gamma *self.update_params["v_b"+str(i+1)] + eta * (self.dB[i+1]/mini_batch_size)
                        self.params["W"+str(i+1)] -= (self.update_params["v_w"+str(i+1)] + eta*wt_decay*(self.dW[i+1]/mini_batch_size))
                        self.params["B"+str(i+1)] -= self.update_params["v_b"+str(i+1)]
                #if display_loss:
                Y_pred = self.predict(X)
                Y_pred_val = self.predict(X_VAL)
                Y_temp = np.argmax(Y_pred,1)
                Y_val_temp = np.argmax(Y_pred_val,1)
                train_acc[epoch] = accuracy_score(y_train, Y_temp)
                val_acc[epoch] = accuracy_score(y_val, Y_val_temp)

                full_output = self.forward_pass(X)
                cost = self.compute_loss(Y, full_output, M, wt_decay)
                train_loss[epoch] = cost
                
                # Mean loss for the validation set
                out = self.forward_pass(X_VAL)
                val_cost = self.compute_loss(Y_VAL, out, Mval, wt_decay)
                val_loss[epoch] = val_cost

                        
        
        elif self.algo == "RMS_Prop":
            for epoch in range(epochs):
                m = X.shape[0]
                for k in range(0,m,mini_batch_size):
                    self.grad(X[k:k+mini_batch_size], Y[k:k+mini_batch_size])
                #self.grad(X, Y)
                    for i in range(self.nh+1):
                        self.update_params["v_w"+str(i+1)] = beta*self.update_params["v_w"+str(i+1)] +(1-beta)*((self.dW[i+1]/mini_batch_size)**2)
                        self.update_params["v_b"+str(i+1)] = beta*self.update_params["v_b"+str(i+1)] +(1-beta)*((self.dB[i+1]/mini_batch_size)**2)
                        self.params["W"+str(i+1)] -= ((eta/(np.sqrt(self.update_params["v_w"+str(i+1)]+eps)))*(self.dW[i+1]/mini_batch_size) + eta*wt_decay*(self.dW[i+1]/mini_batch_size))
                        self.params["B"+str(i+1)] -= (eta/(np.sqrt(self.update_params["v_b"+str(i+1)]+eps)))*(self.dB[i+1]/mini_batch_size)
                Y_pred = self.predict(X)
                Y_pred_val = self.predict(X_VAL)
                Y_temp = np.argmax(Y_pred,1)
                Y_val_temp = np.argmax(Y_pred_val,1)
                train_acc[epoch] = accuracy_score(y_train, Y_temp)
                val_acc[epoch] = accuracy_score(y_val, Y_val_temp)
                full_output = self.forward_pass(X)
                cost = self.compute_loss(Y, full_output, M, wt_decay)
                train_loss[epoch] = cost
                
                # Mean loss for the validation set
                out = self.forward_pass(X_VAL)
                val_cost = self.compute_loss(Y_VAL, out, Mval, wt_decay)
                val_loss[epoch] = val_cost

                        
        elif self.algo == "NAG":
            for epoch in range(epochs):
                m = X.shape[0]
                temp_params = {}
                
                for k in range(0,m,mini_batch_size):
                    for i in range(self.nh+1):
                        self.update_params["v_w"+str(i+1)]=gamma*self.prev_update_params["v_w"+str(i+1)]
                        self.update_params["v_b"+str(i+1)]=gamma*self.prev_update_params["v_b"+str(i+1)]
                        temp_params["W"+str(i+1)]=self.params["W"+str(i+1)]-self.update_params["v_w"+str(i+1)]
                        temp_params["B"+str(i+1)]=self.params["B"+str(i+1)]-self.update_params["v_b"+str(i+1)]
                    self.grad(X[k:k+mini_batch_size], Y[k:k+mini_batch_size],temp_params)
                    for i in range(self.nh+1):
                        self.update_params["v_w"+str(i+1)] = gamma *self.update_params["v_w"+str(i+1)] + eta * (self.dW[i+1]/mini_batch_size)
                        self.update_params["v_b"+str(i+1)] = gamma *self.update_params["v_b"+str(i+1)] + eta * (self.dB[i+1]/mini_batch_size)
                        self.params["W"+str(i+1)] -= (eta * (self.update_params["v_w"+str(i+1)]) + eta*wt_decay*(self.dW[i+1]/mini_batch_size))
                        self.params["B"+str(i+1)] -= eta * (self.update_params["v_b"+str(i+1)])
                    self.prev_update_params=self.update_params 
                Y_pred = self.predict(X)
                Y_pred_val = self.predict(X_VAL)
                Y_temp = np.argmax(Y_pred,1)
                Y_val_temp = np.argmax(Y_pred_val,1)
                train_acc[epoch] = accuracy_score(y_train, Y_temp)
                val_acc[epoch] = accuracy_score(y_val, Y_val_temp)
                full_output = self.forward_pass(X)
                cost = self.compute_loss(Y, full_output, M, wt_decay)
                train_loss[epoch] = cost
                
                # Mean loss for the validation set
                out = self.forward_pass(X_VAL)
                val_cost = self.compute_loss(Y_VAL, out, Mval, wt_decay)
                val_loss[epoch] = val_cost

                    
        elif self.algo == "Adam":
            for epoch in range(epochs):
                m = X.shape[0]
                for k in range(0,m,mini_batch_size):
                    self.grad(X[k:k+mini_batch_size], Y[k:k+mini_batch_size])
                    num_updates=0
                    for i in range(self.nh+1):
                        num_updates+=1
                        self.update_params["m_w"+str(i+1)]=beta1*self.update_params["m_w"+str(i+1)]+(1-beta1)*(self.dW[i+1]/mini_batch_size)
                        self.update_params["v_w"+str(i+1)]=beta2*self.update_params["v_w"+str(i+1)]+(1-beta2)*((self.dW[i+1]/mini_batch_size)**2)
                        m_w_hat=self.update_params["m_w"+str(i+1)]/(1-np.power(beta1,num_updates))
                        v_w_hat=self.update_params["v_w"+str(i+1)]/(1-np.power(beta2,num_updates))
                        self.params["W"+str(i+1)] -=((eta/np.sqrt(v_w_hat+eps))*m_w_hat + eta*wt_decay*(self.dW[i+1]/mini_batch_size))

                        self.update_params["m_b"+str(i+1)]=beta1*self.update_params["m_b"+str(i+1)]+(1-beta1)*(self.dB[i+1]/mini_batch_size)
                        self.update_params["v_b"+str(i+1)]=beta2*self.update_params["v_b"+str(i+1)]+(1-beta2)*((self.dB[i+1]/mini_batch_size)**2)
                        m_b_hat=self.update_params["m_b"+str(i+1)]/(1-np.power(beta1,num_updates))
                        v_b_hat=self.update_params["v_b"+str(i+1)]/(1-np.power(beta2,num_updates))
                        self.params["B"+str(i+1)] -=(eta/np.sqrt(v_b_hat+eps))*m_b_hat
                
                Y_pred = self.predict(X)
                Y_pred_val = self.predict(X_VAL)
                Y_temp = np.argmax(Y_pred,1)
                Y_val_temp = np.argmax(Y_pred_val,1)
                train_acc[epoch] = accuracy_score(y_train, Y_temp)
                val_acc[epoch] = accuracy_score(y_val, Y_val_temp)

                full_output = self.forward_pass(X)
                cost = self.compute_loss(Y, full_output, M, wt_decay)
                train_loss[epoch] = cost
                
                # Mean loss for the validation set
                out = self.forward_pass(X_VAL)
                val_cost = self.compute_loss(Y_VAL, out, Mval, wt_decay)
                val_loss[epoch] = val_cost

                        
        elif self.algo == "NAdam":
            for epoch in range(epochs):
                m = X.shape[0]
                for k in range(0,m,mini_batch_size):
                    self.grad(X[k:k+mini_batch_size], Y[k:k+mini_batch_size])
                    num_updates=0
                    for i in range(self.nh+1):
                        num_updates+=1
                        self.update_params["m_w"+str(i+1)]=beta1*self.update_params["m_w"+str(i+1)]+(1-beta1)*(self.dW[i+1]/mini_batch_size)
                        self.update_params["v_w"+str(i+1)]=beta2*self.update_params["v_w"+str(i+1)]+(1-beta2)*((self.dW[i+1]/mini_batch_size)**2)
                        m_w_hat=self.update_params["m_w"+str(i+1)]/(1-np.power(beta1,num_updates))
                        v_w_hat=self.update_params["v_w"+str(i+1)]/(1-np.power(beta2,num_updates))
                        self.params["W"+str(i+1)] -=((eta/np.sqrt(v_w_hat+eps))*((beta1*m_w_hat + (1-beta1)*v_w_hat)/(1-beta1)) + eta*wt_decay*(self.dW[i+1]/mini_batch_size))

                        self.update_params["m_b"+str(i+1)]=beta1*self.update_params["m_b"+str(i+1)]+(1-beta1)*(self.dB[i+1]/mini_batch_size)
                        self.update_params["v_b"+str(i+1)]=beta2*self.update_params["v_b"+str(i+1)]+(1-beta2)*((self.dB[i+1]/mini_batch_size)**2)
                        m_b_hat=self.update_params["m_b"+str(i+1)]/(1-np.power(beta1,num_updates))
                        v_b_hat=self.update_params["v_b"+str(i+1)]/(1-np.power(beta2,num_updates))
                        self.params["B"+str(i+1)] -=(eta/np.sqrt(v_b_hat+eps))*((beta1*m_b_hat + (1-beta1)*v_b_hat)/(1-beta1))
                Y_pred = self.predict(X)
                Y_pred_val = self.predict(X_VAL)
                Y_temp = np.argmax(Y_pred,1)
                Y_val_temp = np.argmax(Y_pred_val,1)
                train_acc[epoch] = accuracy_score(y_train, Y_temp)
                val_acc[epoch] = accuracy_score(y_val, Y_val_temp)
                full_output = self.forward_pass(X)
                cost = self.compute_loss(Y, full_output, M, wt_decay)
                train_loss[epoch] = cost
                
                # Mean loss for the validation set
                out = self.forward_pass(X_VAL)
                val_cost = self.compute_loss(Y_VAL, out, Mval, wt_decay)
                val_loss[epoch] = val_cost


        #self.plot_loss_curve(train_loss, val_loss)            
        
        return train_acc,val_acc,train_loss,val_loss

Loading MNIST data:

In [12]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [13]:
x_train = x_train / 255.0
x_test = x_test / 255.0
x_train = x_train.reshape(len(x_train),784)
x_test = x_test.reshape(len(x_test),784)

In [14]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=1)

In [15]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(54000, 784)
(54000,)
(6000, 784)
(6000,)


In [16]:
enc = OneHotEncoder()
y_OHE_train = enc.fit_transform(np.expand_dims(y_train,1)).toarray()
y_OHE_val = enc.fit_transform(np.expand_dims(y_val,1)).toarray()
y_OHE_test = enc.fit_transform(np.expand_dims(y_test,1)).toarray()
print(y_OHE_train.shape, y_OHE_val.shape, y_OHE_test.shape)

(54000, 10) (6000, 10) (10000, 10)


Configuration 1:


In [17]:
ffnn = FFNNClass(hidden_sizes=[64,64,64],algo = 'Adam',initialization = 'xavier', activation = 'relu', loss = 'cross_entropy')
a,b,c,d = ffnn.fit(x_train,y_OHE_train,x_val,y_OHE_val, epochs=10, eta=0.001,mini_batch_size=64,wt_decay=0.0005)

In [18]:
y_pred_test1 = ffnn.predict(x_test)
y_pred_test1 = np.argmax(y_pred_test1,1)
test_accuracy1 = accuracy_score(y_test, y_pred_test1)
print("Testing accuracy", round(test_accuracy1, 3))

Testing accuracy 0.973


Configuration 2:

In [19]:
ffnn = FFNNClass(hidden_sizes=[128,128,128],algo = 'NAdam',initialization = 'xavier', activation = 'relu', loss = 'cross_entropy')
a,b,c,d = ffnn.fit(x_train,y_OHE_train,x_val,y_OHE_val, epochs=10, eta=0.0001,mini_batch_size=64,wt_decay=0.0005)

In [20]:
y_pred_test2 = ffnn.predict(x_test)
y_pred_test2 = np.argmax(y_pred_test2,1)
test_accuracy2 = accuracy_score(y_test, y_pred_test2)
print("Testing accuracy", round(test_accuracy2, 3))

Testing accuracy 0.975


Configuration 3:

In [21]:
ffnn = FFNNClass(hidden_sizes=[128,128,128],algo = 'Adam',initialization = 'xavier', activation = 'tanh', loss = 'cross_entropy')
a,b,c,d = ffnn.fit(x_train,y_OHE_train,x_val,y_OHE_val, epochs=10, eta=0.0001,mini_batch_size=32,wt_decay=0)

In [22]:
y_pred_test3 = ffnn.predict(x_test)
y_pred_test3 = np.argmax(y_pred_test3,1)
test_accuracy3 = accuracy_score(y_test, y_pred_test3)
print("Testing accuracy", round(test_accuracy3, 3))

Testing accuracy 0.969
